# Read Noise Distributions

<br>Owner: **Emily Phillips Longley** and **Keith Bechtol**
<br>Last Verified to Run: **2018-11-14**
<br>Verified Stack Release: **w_2018_45**

See examples in the eotest:
* [read_noise.py](https://github.com/lsst-camera-dh/eotest/blob/master/python/lsst/eotest/sensor/read_noise.py)
* [total_noise_histograms.py](https://github.com/lsst-camera-dh/eotest/blob/master/python/lsst/eotest/sensor/total_noise_histograms.py)

Our goal is to compute determine the noise distributions for each amplifier. The total noise has contributions from read noise, system noise, and shot noise added in quadrature.

$\sigma_{\rm total} = \sqrt{\sigma_{\rm read}^{2} + \sigma_{\rm system}^{2} + \sigma_{\rm shot}^{2}}$

**Read noise:**

**System noise:**

**Total read noise:**

**Shot noise:** Poisson noise contribution from dark current. The variance is equal to the dark current (electrons per second) multiplied by the exposure time (seconds).

Other things to check. Look at adjascent pixels. Correlations. Some will be correlated, some will be uncorrelated.

Inputs include the 
* gain per amp, 
* dark current array over pixels

In [ ]:
! eups list -s | grep lsst_distrib

In [ ]:
%matplotlib inline

# system imports
from matplotlib import pylab as plt

# LSST stack imports
from lsst.daf.persistence import Butler
from lsst.ip.isr import IsrTask

In [ ]:
isr_config = IsrTask.ConfigClass()

isr_config.doDark=False
isr_config.doFlat=False
isr_config.doFringe=False
isr_config.doDefect=False
isr_config.doAddDistortionModel=False
isr_config.doLinearize=False

In [ ]:
isr = IsrTask(config=isr_config)

In [ ]:
# Instantiate butler and print out what types of raw exposures exist in this data repository
BOOTCAMP_REPO_DIR = '/project/bootcamp/repo_RTM-007/'
butler = Butler(BOOTCAMP_REPO_DIR)
butler.queryMetadata('raw', ['imageType'])

## Read noise

In [ ]:
# Start by identifying one bias frame
visits = butler.queryMetadata('raw', ['visit'], dataId={'imageType': 'BIAS'})
print(visits)

In [ ]:
dId = dict(visit=visits[0], detector=2) # Note that detector is a required keyword
print(dId)

In [ ]:
raw = butler.get('raw', dId)

# Show metadata for this frame
# print(raw.getMetadata())

# Show the detector name
# raw.getDetector().getName()

In [ ]:
# Get the corresponding master bias and master dark
bias = butler.get('bias', dId)
dark = butler.get('dark', dId)

In [ ]:
result = isr.run(raw, bias=bias)

In [ ]:
image = result.exposure.getMaskedImage().getImage().getArray()

plt.figure()
plt.hist(image)

In [ ]:
butler.queryMetadata('raw', ['detector', 'detectorName'])

In [ ]:


butler.getKeys('bias')
butler.queryMetadata('bias', ['detector'])
#bias = butler.queryMetadata('bias', ['visit'])
#print(bias)

In [ ]:
dId = dict(visit=visits[0], detector=2)
raw = butler.get('raw', dId, visit=258345990)

In [ ]:
dark = butler.get('dark', dId)
dark.getMetadata().toDict()

for a in raw.getDetector():
    print(a.getName(), a.getReadNoise(), a.getGain())

In [ ]:
result = isr.run(raw, bias=bias)

## Creating master dark

---

In [ ]:
%matplotlib inline

# system imports
from matplotlib import pylab as plt
import numpy
import os

# LSST stack imports
from lsst.daf.persistence import Butler
import lsst.afw.display as afwDisplay

In [ ]:
import matplotlib
matplotlib.rcParams['figure.dpi'] = 120

This notebook requires the package `obs_lsst`, which is not currently distributed in the LSP. If the following cell raises an exception, see the instructions in `welcome_to_FE55.ipynb`.

In [ ]:
import eups
assert eups.getSetupVersion("obs_lsst")

In [ ]:
my_channel = '{}_test_channel'.format(os.environ['USER'])
server = 'https://lsst-lspdev.ncsa.illinois.edu'


afwDisplay.setDefaultBackend('firefly')
afw_display = afwDisplay.getDisplay(frame=1, 
                                    name=my_channel)

In [ ]:
BOOTCAMP_REPO_DIR= '/project/bootcamp/repo_RTM-007/'
butler = Butler(BOOTCAMP_REPO_DIR)
visits = butler.queryMetadata('raw', ['visit'], dataId={'imageType': 'FLAT', 'testType': 'FLAT'})

In [ ]:
dId = {'visit': visits[0], 'detector': 2}
raw = butler.get('raw', **dId)

In [ ]:
detector = raw.getDetector()

In [ ]:
afw_display.mtv(raw)

In [ ]:
# for the geometry we've chosen, the y=0 amps are in the top
# Normally the read corner would tell you this, but there is currently a bug
flipXY = {'C00':(False, True),
          'C01':(False, True),
          'C02':(False, True),
          'C03':(False, True),
          'C04':(False, True),
          'C05':(False, True),
          'C06':(False, True),
          'C07':(False, True),
          'C10':(True, False),
          'C11':(True, False),
          'C12':(True, False),
          'C13':(True, False),
          'C14':(True, False),
          'C15':(True, False),
          'C16':(True, False),
          'C17':(True, False)}

In [ ]:
overscans = {}
for key, value in flipXY.items():
    bbox = detector[key].getRawHorizontalOverscanBBox()
    overscans[key] = raw[bbox].getMaskedImage().getImage().clone() # since these are views into the pixels, we want a copy so we don't mess with the pixels in the original image

In [ ]:
for channel_name, flips in flipXY.items():
    # get all the overscans in the same orientation and plot the mean along the y-axis
    arr = overscans[channel_name].getArray() # pull out numpy array
    oscan = numpy.mean(arr)
    fx, fy = flips
    if fx:
        arr = numpy.flip(arr, axis=1)
    if fy:
        arr = numpy.flip(arr, axis=0)
    # aggregate along y-axis.  It's ok to use array math here since we don't have any masks to begin with.
    mean = numpy.mean(arr, axis=0)
    plt.plot(mean-oscan, label=channel_name)

plt.xlabel('pixel index')
plt.ylabel('mean counts along parallel direction')
plt.xlim(0, 65)
plt.ylim(-0.5, 0.5)
plt.legend()